In [33]:
import os
import pandas as pd
import numpy as np
import librosa, librosa.display
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import random

import torch 
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision.transforms import transforms

In [34]:
meta = pd.read_csv(r'C:\Users\sally\Documents\Fall 2020\CIS 519 - Intro to Machine Learning\Project\audioclassification_meta.csv')
meta

,VoxCeleb1 ID,VGGFace1 ID,Gender,Nationality,Set
0,id10001,A.J._Buckley,m,Ireland,dev
1,id10002,A.R._Rahman,m,India,dev
2,id10003,Aamir_Khan,m,India,dev
3,id10004,Aaron_Tveit,m,USA,dev
4,id10005,Aaron_Yoo,m,USA,dev
...,...,...,...,...,...
1206,id11247,Zachary_Levi,m,USA,dev
1207,id11248,Zachary_Quinto,m,USA,dev
1208,id11249,Zack_Snyder,m,USA,dev
1209,id11250,Zoe_Saldana,f,USA,dev


In [35]:
directory = r'C:\Users\sally\Documents\vox1_dev_wav\wav'
filenames = []
for foldername in os.listdir(directory):
    folder_dir = os.path.join(directory,foldername)
    for subfoldername in os.listdir(folder_dir):
        subfolder_dir = os.path.join(folder_dir,subfoldername)
        for filename in os.listdir(subfolder_dir):
            file = os.path.join(subfolder_dir,filename)
            filenames.append((foldername,file))

In [36]:
files = pd.DataFrame(filenames)
files.rename(columns={0:'ID',1:'file'},inplace=True)

In [37]:
wav_df = meta.merge(files,left_on='VoxCeleb1 ID',right_on='ID')[['ID','Gender','Nationality','file']]

In [38]:
wav_df

,ID,Gender,Nationality,file
0,id10001,m,Ireland,C:\Users\sally\Documents\vox1_dev_wav\wav\id10...
1,id10001,m,Ireland,C:\Users\sally\Documents\vox1_dev_wav\wav\id10...
2,id10001,m,Ireland,C:\Users\sally\Documents\vox1_dev_wav\wav\id10...
3,id10001,m,Ireland,C:\Users\sally\Documents\vox1_dev_wav\wav\id10...
4,id10001,m,Ireland,C:\Users\sally\Documents\vox1_dev_wav\wav\id10...
...,...,...,...,...
148637,id11251,f,USA,C:\Users\sally\Documents\vox1_dev_wav\wav\id11...
148638,id11251,f,USA,C:\Users\sally\Documents\vox1_dev_wav\wav\id11...
148639,id11251,f,USA,C:\Users\sally\Documents\vox1_dev_wav\wav\id11...
148640,id11251,f,USA,C:\Users\sally\Documents\vox1_dev_wav\wav\id11...


### Get Samples

In [39]:
# Get list of nationalities
listNat=wav_df['Nationality'].unique()

In [40]:
# Cap samples at 1000 max
columns=['ID', 'Gender', 'Nationality','file']
samples = pd.DataFrame(columns=columns)
for i in range(len(listNat)):
    total = wav_df[wav_df['Nationality'] == listNat[i]].count()['ID']
    num = int(0.5 * total)
    if num < 500:
        samp = wav_df[wav_df['Nationality'] == listNat[i]].sample(num,random_state=42)
    else:
        samp = wav_df[wav_df['Nationality'] == listNat[i]].sample(500,random_state=42)
    samples = samples.append(samp)

In [41]:
samples.groupby('Nationality').nunique()

,ID,Gender,Nationality,file
Nationality,,,,
Australia,37,2,1,500
Austria,1,1,1,139
Brazil,1,1,1,33
Canada,52,2,1,500
Chile,3,2,1,91
China,2,2,1,188
Croatia,3,2,1,144
Denmark,3,2,1,141
Germany,9,2,1,500


### Feature Extraction

In [ ]:
# Unused
def extract_features(signal, sr, n_fft, hop_length, n_mfcc):    
    # Extract short-time fourier transform
    stft = librosa.core.stft(signal, hop_length=hop_length, n_fft=n_fft)
    
    # Extract log spectrogram
    log_spectrogram = extract_spectrogram(stft)
    
    # Extract MFCC
    MFCC = librosa.feature.mfcc(signal, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc)
    
    # Extract chromagram
    chromagram = librosa.feature.chroma_stft(signal, sr=sr, hop_length=hop_length)
    
    # Extract harmonics and percussion
    harmonics, percussion = extract_harmonics_percussion(stft)
    
    return log_spectrogram, MFCC, chromagram, harmonics, percussion

In [42]:
# Filter frequency using FFT
def filter_signal(signal):
    # Take the Fourier transform of the data
    F = np.fft.fft(signal)

    # Filter out any with magnitude < 20
    F_filtered = np.array([0.0 if np.abs(x) < 20 else x for x in F])

    # Reconstruct the filtered signal
    filtered_signal = np.fft.ifft(F_filtered)
    filtered_signal = np.array([float(x) for x in filtered_signal])
    
    return filtered_signal

In [43]:
def extract_spectrogram(stft):
    spectrogram = np.abs(stft)
    log_spectrogram = librosa.amplitude_to_db(spectrogram) #amplitude as a function of time and frequency
    
    return log_spectrogram

In [ ]:
def extract_harmonics_percussion(stft):
    harm, perc = librosa.decompose.hpss(stft)
    harm = librosa.amplitude_to_db(np.abs(harm))
    perc = librosa.amplitude_to_db(np.abs(perc))
    
    return harm, perc

In [44]:
def get_spectrogram(file, sr, n_fft, hop_length):
    # Get signal from file
    signal, sampling_rate = librosa.load(file, sr=sr, duration=3)
    
    # Filter out noise
    filt_signal = filter_signal(signal)
    
    # Extract short-time fourier transform
    stft = librosa.core.stft(filt_signal, hop_length=hop_length, n_fft=n_fft)
    
    # Extract log spectrogram
    log_spectrogram = extract_spectrogram(stft)
    
    return log_spectrogram

In [48]:
def get_MFCC(file, sr, n_fft, hop_length, n_mfcc):
    # Get signal from file
    signal, sampling_rate = librosa.load(file, sr=sr, duration=3)
    
    # Filter out noise
    filt_signal = filter_signal(signal)
    
    # Extract MFCC
    MFCC = librosa.feature.mfcc(filt_signal, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc)
    
    return MFCC

In [46]:
# Define features
sr=8000                  # sampling rate
n_fft=2048               # number of samples
hop_length=512           # amount we shift each fourier transfer to the right
n_mfcc=13                # number of MFCCs to extract

In [ ]:
labels = np.array(samples['Nationality'])
samples['Gender'] = samples['Gender'].apply(lambda x: 0 if x=='m' else 1)

# Extract spectrograms to start
features_spectrogram = samples['file'].apply(lambda x: get_spectrogram(x,sr,n_fft,hop_length))

In [49]:
# Extract MFCCs
features_MFCC = samples['file'].apply(lambda x: get_MFCC(x,sr,n_fft,hop_length,n_mfcc))

<ipython-input-42-0e00b1ce7fcd>:11: ComplexWarning: Casting complex values to real discards the imaginary part
  filtered_signal = np.array([float(x) for x in filtered_signal])


In [ ]:
# Cast spectrogram series into array
arr_features_spectrogram = np.array(features_spectrogram)

# Save spectrograms
np.save('features_spectrogram.npy', arr_features_spectrogram)
np.save('labels.npy', labels)

In [50]:
# Cast MFCCs series into array
arr_features_MFCCs = np.array(features_MFCC)

# Save MFCCs
np.save('features_MFCC.npy', arr_features_MFCCs)

### Modelling

In [77]:
# Reload features when needed
features=np.load('features_spectrogram.npy',allow_pickle=True)
# features=np.load('features_MFCC.npy',allow_pickle=True)
labels_str=np.load('labels.npy',allow_pickle=True)

In [78]:
# Turn labels into numeric values, create dictionary to map back later 
le = preprocessing.LabelEncoder()
le.fit(labels_str)
labels=le.transform(labels_str)

In [79]:
# Just test on English speaking countries for now
eng_countries=[0,3,12,14,17,33,34]
features_eng = []
labels_eng = []
for i in range(len(labels)):
    if labels[i] in eng_countries:
        features_eng.append(features[i])
        labels_eng.append(labels[i])

features_eng = np.array(features_eng)
labels_eng = np.array(labels_eng)

In [80]:
# Randomly drop 10 to make batch sizes even
drop = random.sample(range(0,features_eng.shape[0]),3200)
features_eng=features_eng[list(drop)]
labels_eng=labels_eng[list(drop)]

In [81]:
# Cast array to type float64 instead of object for tensors to work
features_eng = [np.array(list(x),dtype=np.float64) for x in features_eng]
features_eng = np.array(features_eng,dtype=np.float64)

labels_eng = [np.long(x) for x in labels_eng]
labels_eng = np.array(labels_eng,dtype=np.long)

In [82]:
# Get train and test datasets
X_train, X_test, y_train, y_test = train_test_split(features_eng, labels_eng, test_size=0.3, random_state=42)

In [83]:
X_train.shape

(2240, 1025, 47)

In [76]:
class Convolutional(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=1,out_channels=10,kernel_size=4,stride=1,padding=0)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=4,stride=2)
        
        self.conv2 = torch.nn.Conv2d(in_channels=10, out_channels=32, kernel_size=4, stride=1, padding=0)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=4,stride=2)
        
        self.fc1 = torch.nn.Linear(in_features=32*252*8,out_features=1000)
        self.drop1 = torch.nn.Dropout(0.2)
        self.fc2 = torch.nn.Linear(in_features=1000,out_features=500)
        self.drop2 = torch.nn.Dropout(0.2)
        self.fc3 = torch.nn.Linear(in_features=500,out_features=100)


    def forward(self, X):
        batch_size = 64
        X = self.conv1(X)
        X = self.pool1(X)
        X = self.conv2(X)
        X = self.pool2(X)
        X = X.relu()
        X = X.view(batch_size, -1)
        X = self.fc1(X)
        X - self.drop1(X)
        X = X.relu()
        X = self.fc2(X)
        X - self.drop2(X)
        X = X.relu()
        X = self.fc3(X)

        return X

In [72]:
class Convolutional_MFCC(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=1,out_channels=10,kernel_size=3,stride=1,padding=0)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=3,stride=2)
        
        self.conv2 = torch.nn.Conv2d(in_channels=10, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=3,stride=2)
        
        self.fc1 = torch.nn.Linear(in_features=32*3*3,out_features=100)
        self.drop1 = torch.nn.Dropout(0.2)
        self.fc2 = torch.nn.Linear(in_features=100,out_features=50)


    def forward(self, X):
        batch_size = 64
        X = self.conv1(X)
        X = self.pool1(X)
        X = self.conv2(X)
        X = self.pool2(X)
        X = X.relu()
        X = X.view(batch_size, -1)
        X = self.fc1(X)
        X - self.drop1(X)
        X = X.relu()
        X = self.fc2(X)

        return X

In [29]:
def compute_loss_and_accuracy(network, data_loader):
    total_loss = 0
    num_correct = 0
    num_instances = 0

    cross_entropy_loss = torch.nn.CrossEntropyLoss()

    for X, y in data_loader:
        with torch.no_grad():
            y_pred = network(X)
            total_loss += cross_entropy_loss(y_pred,y).item() * X.size(0)

        for i in range(len(y_pred)):
            predicted = torch.argmax(y_pred[i])
            actual = y[i]

            if predicted == actual:
                num_correct += 1

        num_instances += X.size(0)
  
    accuracy = num_correct / num_instances * 100
    average_loss = total_loss / num_instances

    return accuracy, average_loss

In [84]:
def run_experiment(network, train_data_loader, valid_data_loader, optimizer):
    train_losses = []
    valid_accs = []

    cross_entropy_loss = torch.nn.CrossEntropyLoss()

    for epoch in range(100):
        print('Epoch: ' + str(epoch))
        total_loss = 0.0
        num_instances = 0

        for X, y in train_data_loader:
            optimizer.zero_grad()
            y_pred = network(X)

            loss = cross_entropy_loss(y_pred,y)
            total_loss+=loss.item() * X.size(0)
            loss.backward()

            optimizer.step()

            num_instances += X.size(0)

        train_loss = total_loss / num_instances
        valid_acc, _ = compute_loss_and_accuracy(network, valid_data_loader)
        print(valid_acc)

        train_losses.append(train_loss)
        valid_accs.append(valid_acc)
    return train_losses, valid_accs

In [85]:
# Initialize tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float)
X_train_tensor = X_train_tensor.reshape([X_train.shape[0],1,X_train.shape[1],X_train.shape[2]])
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float)
X_test_tensor = X_test_tensor.reshape([X_test.shape[0],1,X_test.shape[1],X_test.shape[2]])
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_data_loader = DataLoader(train_dataset, batch_size=64,shuffle=True)

valid_dataset = TensorDataset(X_test_tensor, y_test_tensor)
valid_data_loader = DataLoader(valid_dataset, batch_size=64)

In [86]:
X_train_tensor.shape

torch.Size([2240, 1, 1025, 47])

In [87]:
network = Convolutional()
sgd = torch.optim.SGD(network.parameters(), lr=0.001)

train_losses, valid_accs = run_experiment(network, train_data_loader, valid_data_loader, sgd)

Epoch: 0
18.333333333333332
Epoch: 1
17.8125
Epoch: 2
15.937499999999998
Epoch: 3
18.333333333333332
Epoch: 4
21.25
Epoch: 5
21.145833333333332
Epoch: 6
20.3125
Epoch: 7
19.6875
Epoch: 8
21.666666666666668
Epoch: 9
15.729166666666666
Epoch: 10
22.083333333333332
Epoch: 11
17.291666666666668
Epoch: 12
20.520833333333332
Epoch: 13
22.5
Epoch: 14
20.729166666666668
Epoch: 15
21.354166666666664
Epoch: 16
22.8125
Epoch: 17
22.708333333333332
Epoch: 18
20.520833333333332
Epoch: 19
21.979166666666668
Epoch: 20
22.395833333333336
Epoch: 21
24.166666666666668
Epoch: 22
20.625
Epoch: 23
23.229166666666668
Epoch: 24
23.541666666666668
Epoch: 25
24.479166666666664
Epoch: 26
22.604166666666668
Epoch: 27
22.5
Epoch: 28
22.1875
Epoch: 29
22.395833333333336
Epoch: 30
24.0625
Epoch: 31
20.9375
Epoch: 32
23.541666666666668
Epoch: 33
23.541666666666668
Epoch: 34
23.645833333333332
Epoch: 35
25.0
Epoch: 36
22.916666666666664
Epoch: 37
22.395833333333336
Epoch: 38
24.0625
Epoch: 39
20.416666666666668
Epoch

In [75]:
# MFCC
network_mfcc = Convolutional_MFCC()
sgd = torch.optim.SGD(network_mfcc.parameters(), lr=0.001)

train_losses, valid_accs = run_experiment(network_mfcc, train_data_loader, valid_data_loader, sgd)

Epoch: 0
15.312500000000002
Epoch: 1
17.916666666666668
Epoch: 2
14.583333333333334
Epoch: 3
18.333333333333332
Epoch: 4
19.791666666666664
Epoch: 5
16.354166666666668
Epoch: 6
18.645833333333332
Epoch: 7
17.1875
Epoch: 8
17.916666666666668
Epoch: 9
18.75
Epoch: 10
20.208333333333332
Epoch: 11
20.833333333333336
Epoch: 12
18.958333333333332
Epoch: 13
19.0625
Epoch: 14
20.833333333333336
Epoch: 15
18.020833333333332
Epoch: 16
18.75
Epoch: 17
20.104166666666668
Epoch: 18
20.104166666666668
Epoch: 19
20.416666666666668
Epoch: 20
18.958333333333332
Epoch: 21
20.416666666666668
Epoch: 22
21.145833333333332
Epoch: 23
21.770833333333332
Epoch: 24
21.666666666666668
Epoch: 25
20.625
Epoch: 26
23.75
Epoch: 27
21.5625
Epoch: 28
18.333333333333332
Epoch: 29
20.833333333333336
Epoch: 30
20.729166666666668
Epoch: 31
20.833333333333336
Epoch: 32
22.604166666666668
Epoch: 33
21.979166666666668
Epoch: 34
21.25
Epoch: 35
22.291666666666668
Epoch: 36
22.8125
Epoch: 37
22.708333333333332
Epoch: 38
21.770

21.458333333333332
Epoch: 318
21.875
Epoch: 319
19.375
Epoch: 320
20.9375
Epoch: 321
19.791666666666664
Epoch: 322
20.3125
Epoch: 323
20.416666666666668
Epoch: 324
19.791666666666664
Epoch: 325
22.1875
Epoch: 326
20.416666666666668
Epoch: 327
18.854166666666668
Epoch: 328
19.0625
Epoch: 329
19.0625
Epoch: 330
21.25
Epoch: 331
20.0
Epoch: 332
21.979166666666668
Epoch: 333
19.895833333333332
Epoch: 334
18.541666666666668
Epoch: 335
21.5625
Epoch: 336
19.583333333333332
Epoch: 337
20.625
Epoch: 338
18.75
Epoch: 339
21.666666666666668
Epoch: 340
19.479166666666668
Epoch: 341
19.791666666666664
Epoch: 342
20.0
Epoch: 343
21.145833333333332
Epoch: 344
20.520833333333332
Epoch: 345
21.041666666666668
Epoch: 346
20.9375
Epoch: 347
21.041666666666668
Epoch: 348
18.4375
Epoch: 349
20.625
Epoch: 350
19.270833333333336
Epoch: 351
19.583333333333332
Epoch: 352
19.6875
Epoch: 353
21.145833333333332
Epoch: 354
20.520833333333332
Epoch: 355
19.0625
Epoch: 356
18.4375
Epoch: 357
19.375
Epoch: 358
21.56

20.520833333333332
Epoch: 634
19.270833333333336
Epoch: 635
20.520833333333332
Epoch: 636
19.0625
Epoch: 637
18.541666666666668
Epoch: 638
19.6875
Epoch: 639
20.104166666666668
Epoch: 640
20.3125
Epoch: 641
19.583333333333332
Epoch: 642
21.458333333333332
Epoch: 643
19.895833333333332
Epoch: 644
20.104166666666668
Epoch: 645
19.479166666666668
Epoch: 646
20.833333333333336
Epoch: 647
17.916666666666668
Epoch: 648
20.520833333333332
Epoch: 649
19.583333333333332
Epoch: 650
19.6875
Epoch: 651
21.041666666666668
Epoch: 652
20.9375
Epoch: 653
20.208333333333332
Epoch: 654
20.416666666666668
Epoch: 655
19.166666666666668
Epoch: 656
20.416666666666668
Epoch: 657
19.166666666666668
Epoch: 658
19.791666666666664
Epoch: 659
20.833333333333336
Epoch: 660
20.3125
Epoch: 661
18.854166666666668
Epoch: 662
19.791666666666664
Epoch: 663
20.729166666666668
Epoch: 664
20.104166666666668
Epoch: 665
20.0
Epoch: 666
21.25
Epoch: 667
19.791666666666664
Epoch: 668
21.770833333333332
Epoch: 669
19.0625
Epoch

18.958333333333332
Epoch: 949
17.083333333333332
Epoch: 950
19.6875
Epoch: 951
20.833333333333336
Epoch: 952
19.583333333333332
Epoch: 953
20.104166666666668
Epoch: 954
19.895833333333332
Epoch: 955
18.854166666666668
Epoch: 956
18.4375
Epoch: 957
20.208333333333332
Epoch: 958
19.0625
Epoch: 959
20.0
Epoch: 960
18.958333333333332
Epoch: 961
20.416666666666668
Epoch: 962
18.645833333333332
Epoch: 963
19.583333333333332
Epoch: 964
18.75
Epoch: 965
19.6875
Epoch: 966
19.479166666666668
Epoch: 967
20.208333333333332
Epoch: 968
20.520833333333332
Epoch: 969
20.104166666666668
Epoch: 970
20.520833333333332
Epoch: 971
18.645833333333332
Epoch: 972
20.729166666666668
Epoch: 973
18.854166666666668
Epoch: 974
20.104166666666668
Epoch: 975
19.479166666666668
Epoch: 976
20.520833333333332
Epoch: 977
18.958333333333332
Epoch: 978
19.895833333333332
Epoch: 979
18.4375
Epoch: 980
19.791666666666664
Epoch: 981
20.104166666666668
Epoch: 982
18.958333333333332
Epoch: 983
20.104166666666668
Epoch: 984
18